In [19]:
import pandas as pd
import networkx as nx
import igraph as ig
import leidenalg as la

In [ ]:
def read_graph_from_file(file_path):
    df = pd.read_csv(file_path, sep=r'\s+', header=None, names=['FromNode', 'ToNode'])
    
    G = nx.from_pandas_edgelist(df, 'FromNode', 'ToNode')
    return G

In [ ]:
def convert_nx_to_igraph(G):
    mapping = {node: i for i, node in enumerate(G.nodes())}
    G = nx.relabel_nodes(G, mapping) 
    edges = list(G.edges())
    g_igraph = ig.Graph(edges=edges)
    return g_igraph, mapping

In [22]:
def run_leiden_algorithm(g_igraph):
    partition = la.find_partition(g_igraph, la.ModularityVertexPartition)
    communities = [list(community) for community in partition]
    return communities

In [ ]:
def calculate_metrics(G, communities):

    modularity = nx.algorithms.community.modularity(G, communities)
    
    avg_clustering_coeff = nx.average_clustering(G)
    
    num_communities = len(communities)
    
    largest_cc = max(nx.connected_components(G), key=len)
    subgraph = G.subgraph(largest_cc)
    if nx.is_connected(subgraph):
        diameter = nx.diameter(subgraph)
    else:
        diameter = None

    return modularity, avg_clustering_coeff, num_communities, diameter

In [ ]:
def process_file_with_leiden(file_path):
    
    G = read_graph_from_file(file_path)
    g_igraph, mapping = convert_nx_to_igraph(G)
    
    communities = run_leiden_algorithm(g_igraph)
    
    # Mapping the communities back to original node labels
    original_communities = [[list(mapping.keys())[list(mapping.values()).index(node)] for node in community] for community in communities]
    
    modularity, avg_clustering_coeff, num_communities, diameter = calculate_metrics(G, original_communities)
    
    print(f"Modularity: {modularity}")
    print(f"Average Clustering Coefficient: {avg_clustering_coeff}")
    print(f"Number of Communities: {num_communities}")
    if diameter is not None:
        print(f"Diameter of Largest Connected Component: {diameter}")
    else:
        print("Graph is disconnected; diameter is undefined.")

In [ ]:
file_path = r""  # Replace with your actual file path
process_file_with_leiden(file_path)